In [38]:
import os
import io
import glob
import hashlib
import pandas as pd
import xml.etree.ElementTree as ET
import tensorflow as tf
import random

from PIL import Image
import sys
sys.path.insert(0, "./lib/")
import dataset_util


In [39]:
def create_example(xml_file):
    #process the xml file
    tree = ET.parse(xml_file)
    root = tree.getroot()
    image_name = root.find('filename').text
    file_name = image_name.encode('utf8')
    size=root.find('size')
    width = int(size[0].text)
    height = int(size[1].text)
        
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    classes = []
    classes_text = []
    truncated = []
    poses = []
    difficult_obj = []
        
    for member in root.findall('object'):
            
        classes_text.append('Person'.encode('utf8'))
        boundBox = member.find("bndbox")
        xmin.append(float(boundBox[0].text) / width)
        ymin.append(float(boundBox[1].text) / height)
        xmax.append(float(boundBox[2].text) / width)
        ymax.append(float(boundBox[3].text) / height)
        difficult_obj.append(0)
        #if you have more than one classes in dataset you can change the next line
        #to read the class from the xml file and change the class label into its 
        #corresponding integer number, u can use next function structure
        '''
        def class_text_to_int(row_label):
            if row_label == 'Person':
                return 1
            if row_label == 'car':
                return 2
            and so on.....
        '''
        classes.append(1)   # i wrote 1 because i have only one class(person)
        truncated.append(0)
        poses.append('Unspecified'.encode('utf8'))

    #read corresponding image
    full_path = os.path.join('../VOC2012/JPEGImages', '{}'.format(image_name))  #provide the path of images directory
    with tf.gfile.GFile(full_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    if image.format != 'JPEG':
        raise ValueError('Image format not JPEG')
    key = hashlib.sha256(encoded_jpg).hexdigest()

    #create TFRecord Example
    example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(file_name),
        'image/source_id': dataset_util.bytes_feature(file_name),
        'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
        'image/object/difficult': dataset_util.int64_list_feature(difficult_obj),
        'image/object/truncated': dataset_util.int64_list_feature(truncated),
        'image/object/view': dataset_util.bytes_list_feature(poses),
        }))
    return example


In [40]:
def main(_):
    writer_train = tf.python_io.TFRecordWriter('../Tfdata/train.record')     
    writer_test = tf.python_io.TFRecordWriter('../Tfdata/test.record')
    #provide the path to annotation xml files directory
    filename_list=tf.train.match_filenames_once("../VOC2012/Annotations/*.xml")
    init = (tf.global_variables_initializer(), tf.local_variables_initializer())
    sess=tf.Session()
    sess.run(init)
    list=sess.run(filename_list)
    random.shuffle(list)   #shuffle files list
    i=1 
    tst=0   #to count number of images for evaluation 
    trn=0   #to count number of images for training
    for xml_file in list:
        print(xml_file)
        example = create_example(xml_file)
        if (i%10)==0:  #each 10th file (xml and image) write it for evaluation
            writer_test.write(example.SerializeToString())
            tst=tst+1
        else:          #the rest for training
            writer_train.write(example.SerializeToString())
            trn=trn+1
        i=i+1
    writer_test.close()
    writer_train.close()
    print('Successfully converted dataset to TFRecord.')
    print('training dataset: # ')
    print(trn)
    print('test dataset: # ')
    print(tst)

In [13]:
if __name__ == '__main__':
    tf.app.run()

../VOC2012/Annotations/2011_006798.xml
../VOC2012/Annotations/2008_002454.xml
../VOC2012/Annotations/2011_005525.xml
../VOC2012/Annotations/2009_003635.xml
../VOC2012/Annotations/2008_001980.xml
../VOC2012/Annotations/2012_001944.xml
../VOC2012/Annotations/2008_005085.xml
../VOC2012/Annotations/2010_002931.xml
../VOC2012/Annotations/2007_003529.xml
../VOC2012/Annotations/2007_006641.xml
../VOC2012/Annotations/2012_001104.xml
../VOC2012/Annotations/2010_004216.xml
../VOC2012/Annotations/2008_004074.xml
../VOC2012/Annotations/2010_005393.xml
../VOC2012/Annotations/2009_002390.xml
../VOC2012/Annotations/2011_000237.xml
../VOC2012/Annotations/2011_003422.xml
../VOC2012/Annotations/2012_002995.xml
../VOC2012/Annotations/2008_003978.xml
../VOC2012/Annotations/2008_008384.xml
../VOC2012/Annotations/2008_007003.xml
../VOC2012/Annotations/2010_003269.xml
../VOC2012/Annotations/2009_004897.xml
../VOC2012/Annotations/2009_000987.xml
../VOC2012/Annotations/2009_003316.xml
../VOC2012/Annotations/20

../VOC2012/Annotations/2008_000470.xml
../VOC2012/Annotations/2012_001090.xml
../VOC2012/Annotations/2011_003294.xml
../VOC2012/Annotations/2008_001812.xml
../VOC2012/Annotations/2010_005494.xml
../VOC2012/Annotations/2008_001640.xml
../VOC2012/Annotations/2011_004088.xml
../VOC2012/Annotations/2010_006320.xml
../VOC2012/Annotations/2008_002061.xml
../VOC2012/Annotations/2010_000679.xml
../VOC2012/Annotations/2010_006881.xml
../VOC2012/Annotations/2008_006050.xml
../VOC2012/Annotations/2011_005983.xml
../VOC2012/Annotations/2008_007574.xml
../VOC2012/Annotations/2010_006602.xml
../VOC2012/Annotations/2011_001473.xml
../VOC2012/Annotations/2011_006134.xml
../VOC2012/Annotations/2010_006215.xml
../VOC2012/Annotations/2008_006708.xml
../VOC2012/Annotations/2009_001070.xml
../VOC2012/Annotations/2008_000381.xml
../VOC2012/Annotations/2012_001792.xml
../VOC2012/Annotations/2009_002256.xml
../VOC2012/Annotations/2010_004429.xml
../VOC2012/Annotations/2012_003025.xml
../VOC2012/Annotations/20

../VOC2012/Annotations/2010_003976.xml
../VOC2012/Annotations/2009_003588.xml
../VOC2012/Annotations/2009_004624.xml
../VOC2012/Annotations/2012_000715.xml
../VOC2012/Annotations/2012_001410.xml
../VOC2012/Annotations/2011_001624.xml
../VOC2012/Annotations/2011_006046.xml
../VOC2012/Annotations/2012_001874.xml
../VOC2012/Annotations/2010_004948.xml
../VOC2012/Annotations/2007_004476.xml
../VOC2012/Annotations/2009_000320.xml
../VOC2012/Annotations/2009_001873.xml
../VOC2012/Annotations/2010_003807.xml
../VOC2012/Annotations/2008_006743.xml
../VOC2012/Annotations/2008_002080.xml
../VOC2012/Annotations/2011_006806.xml
../VOC2012/Annotations/2011_000239.xml
../VOC2012/Annotations/2011_002300.xml
../VOC2012/Annotations/2011_000749.xml
../VOC2012/Annotations/2007_007531.xml
../VOC2012/Annotations/2009_002112.xml
../VOC2012/Annotations/2010_002211.xml
../VOC2012/Annotations/2007_004328.xml
../VOC2012/Annotations/2011_000872.xml
../VOC2012/Annotations/2010_004847.xml
../VOC2012/Annotations/20

../VOC2012/Annotations/2008_003057.xml
../VOC2012/Annotations/2008_005889.xml
../VOC2012/Annotations/2011_005726.xml
../VOC2012/Annotations/2010_005002.xml
../VOC2012/Annotations/2008_000787.xml
../VOC2012/Annotations/2012_002393.xml
../VOC2012/Annotations/2010_000705.xml
../VOC2012/Annotations/2007_006483.xml
../VOC2012/Annotations/2011_006330.xml
../VOC2012/Annotations/2012_002780.xml
../VOC2012/Annotations/2008_007421.xml
../VOC2012/Annotations/2011_002036.xml
../VOC2012/Annotations/2007_005902.xml
../VOC2012/Annotations/2010_003421.xml
../VOC2012/Annotations/2009_000675.xml
../VOC2012/Annotations/2008_007665.xml
../VOC2012/Annotations/2009_001673.xml
../VOC2012/Annotations/2011_004979.xml
../VOC2012/Annotations/2011_002079.xml
../VOC2012/Annotations/2010_005327.xml
../VOC2012/Annotations/2008_000806.xml
../VOC2012/Annotations/2011_002324.xml
../VOC2012/Annotations/2007_003530.xml
../VOC2012/Annotations/2011_004161.xml
../VOC2012/Annotations/2011_004935.xml
../VOC2012/Annotations/20

../VOC2012/Annotations/2011_007031.xml
../VOC2012/Annotations/2008_007538.xml
../VOC2012/Annotations/2008_004221.xml
../VOC2012/Annotations/2012_002130.xml
../VOC2012/Annotations/2008_008402.xml
../VOC2012/Annotations/2011_001792.xml
../VOC2012/Annotations/2012_003213.xml
../VOC2012/Annotations/2008_002218.xml
../VOC2012/Annotations/2009_004486.xml
../VOC2012/Annotations/2010_003608.xml
../VOC2012/Annotations/2007_005527.xml
../VOC2012/Annotations/2012_003916.xml
../VOC2012/Annotations/2010_006040.xml
../VOC2012/Annotations/2012_000458.xml
../VOC2012/Annotations/2010_002436.xml
../VOC2012/Annotations/2009_000063.xml
../VOC2012/Annotations/2010_003947.xml
../VOC2012/Annotations/2011_005593.xml
../VOC2012/Annotations/2008_002481.xml
../VOC2012/Annotations/2010_000254.xml
../VOC2012/Annotations/2009_003125.xml
../VOC2012/Annotations/2010_003757.xml
../VOC2012/Annotations/2009_001952.xml
../VOC2012/Annotations/2008_007758.xml
../VOC2012/Annotations/2008_003732.xml
../VOC2012/Annotations/20

...

../VOC2012/Annotations/2011_000788.xml
../VOC2012/Annotations/2011_002709.xml
../VOC2012/Annotations/2011_005313.xml
../VOC2012/Annotations/2008_000620.xml
../VOC2012/Annotations/2012_003032.xml
../VOC2012/Annotations/2011_001748.xml
../VOC2012/Annotations/2010_004066.xml
../VOC2012/Annotations/2010_005830.xml
../VOC2012/Annotations/2008_000655.xml
../VOC2012/Annotations/2008_003683.xml
../VOC2012/Annotations/2012_002933.xml
../VOC2012/Annotations/2011_002916.xml
../VOC2012/Annotations/2007_002565.xml
../VOC2012/Annotations/2012_003938.xml
../VOC2012/Annotations/2008_004502.xml
../VOC2012/Annotations/2008_000033.xml
../VOC2012/Annotations/2008_000397.xml
../VOC2012/Annotations/2008_007653.xml
../VOC2012/Annotations/2010_005425.xml
../VOC2012/Annotations/2008_005701.xml
../VOC2012/Annotations/2009_002240.xml
../VOC2012/Annotations/2012_004093.xml
../VOC2012/Annotations/2011_004548.xml
../VOC2012/Annotations/2008_003662.xml
../VOC2012/Annotations/2010_000527.xml
../VOC2012/Annotations/20

../VOC2012/Annotations/2010_001922.xml
../VOC2012/Annotations/2009_003726.xml
../VOC2012/Annotations/2011_000638.xml
../VOC2012/Annotations/2011_006475.xml
../VOC2012/Annotations/2012_002025.xml
../VOC2012/Annotations/2010_005044.xml
../VOC2012/Annotations/2010_002245.xml
../VOC2012/Annotations/2008_000128.xml
../VOC2012/Annotations/2010_004179.xml
../VOC2012/Annotations/2010_002737.xml
../VOC2012/Annotations/2011_004253.xml
../VOC2012/Annotations/2010_000456.xml
../VOC2012/Annotations/2008_008444.xml
../VOC2012/Annotations/2008_004263.xml
../VOC2012/Annotations/2009_001837.xml
../VOC2012/Annotations/2008_007348.xml
../VOC2012/Annotations/2010_004312.xml
../VOC2012/Annotations/2009_003751.xml
../VOC2012/Annotations/2010_004363.xml
../VOC2012/Annotations/2012_003947.xml
../VOC2012/Annotations/2008_006962.xml
../VOC2012/Annotations/2009_002453.xml
../VOC2012/Annotations/2010_004517.xml
../VOC2012/Annotations/2009_003132.xml
../VOC2012/Annotations/2011_004436.xml
../VOC2012/Annotations/20

../VOC2012/Annotations/2010_006839.xml
../VOC2012/Annotations/2011_003662.xml
../VOC2012/Annotations/2009_002732.xml
../VOC2012/Annotations/2009_000408.xml
../VOC2012/Annotations/2007_003367.xml
../VOC2012/Annotations/2010_004278.xml
../VOC2012/Annotations/2008_006818.xml
../VOC2012/Annotations/2011_004765.xml
../VOC2012/Annotations/2011_007037.xml
../VOC2012/Annotations/2010_006135.xml
../VOC2012/Annotations/2012_001997.xml
../VOC2012/Annotations/2009_002406.xml
../VOC2012/Annotations/2008_002709.xml
../VOC2012/Annotations/2009_004448.xml
../VOC2012/Annotations/2010_002534.xml
../VOC2012/Annotations/2010_006340.xml
../VOC2012/Annotations/2008_004653.xml
../VOC2012/Annotations/2012_003662.xml
../VOC2012/Annotations/2011_002098.xml
../VOC2012/Annotations/2011_001764.xml
../VOC2012/Annotations/2011_006390.xml
../VOC2012/Annotations/2008_005373.xml
../VOC2012/Annotations/2008_004684.xml
../VOC2012/Annotations/2011_005171.xml
../VOC2012/Annotations/2007_000830.xml
../VOC2012/Annotations/20

SystemExit: 

/home/mohammad/Tensorflow/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [30]:
data_path = '../Tfdata/train.record'  # address to save the hdf5 file
with tf.Session() as sess:
    feature = {'image/filename': tf.FixedLenFeature([], tf.string),
               'image/object/class/label': tf.FixedLenFeature([], tf.int64)}
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer([data_path], num_epochs=1)
    print(filename_queue)
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['image/filename'], tf.float32)
    
    # Cast label data into int32
    label = tf.cast(features['image/object/class/label'], tf.int32)
    # Reshape image data into the original shape
    image = tf.reshape(image, [224, 224, 3])
    
    print(label)
    print(image)

Tensor("Cast_13:0", shape=(), dtype=int32)
Tensor("Reshape_13:0", shape=(224, 224, 3), dtype=float32)


In [ ]:
tree = ET.parse("../VOC2012/Annotations/2007_000027.xml")
root = tree.getroot()
image_name = root.find('filename').text
file_name = image_name.encode('utf8')
#print(root[2][1][0])
for member in root.findall('object'):
    if member[0].text != "person":
        print("Not person!")
    else:
        boundBox = member.find("bndbox")
        print(boundBox[])
    #print member[5][1][0].text

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
%matplotlib inline

im = np.array(Image.open('./VOC2012/JPEGImages/2011_003617.jpg'), dtype=np.uint8)

# Create figure and axes
fig,ax = plt.subplots(1)

# Display the image
ax.imshow(im)

# Create a Rectangle patch
rect = patches.Rectangle((215,81),(375-81),(487-215),linewidth=1,edgecolor='r',facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

plt.show()

link to load data from pascal VOC: https://gist.github.com/saghiralfasly/ee642af0616461145a9a82d7317fb1d6

In [45]:
from PIL import Image
import numpy as np
import tensorflow as tf

def read_and_decode(filename_queue):
    
 width = int(0)
 height = int(0)
        
 xmin = []
 ymin = []
 xmax = []
 ymax = []
 classes = []
 classes_text = []
 truncated = []
 poses = []
 difficult_obj = []
    
 reader = tf.TFRecordReader()
 _, serialized_example = reader.read(filename_queue)
 features = tf.parse_single_example(
  serialized_example,
  # Defaults are not specified since both keys are required.
  feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(file_name),
        'image/source_id': dataset_util.bytes_feature(file_name),
        'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
        'image/object/difficult': dataset_util.int64_list_feature(difficult_obj),
        'image/object/truncated': dataset_util.int64_list_feature(truncated),
        'image/object/view': dataset_util.bytes_list_feature(poses),
        })
 #image = tf.decode_raw(features['image_raw'], tf.uint8)
 #label = tf.cast(features['label'], tf.int32)
 #height = tf.cast(features['height'], tf.int32)
 #width = tf.cast(features['width'], tf.int32)
 #depth = tf.cast(features['depth'], tf.int32)
 return features


def get_all_records(FILE):
 with tf.Session() as sess:
   filename_queue = tf.train.string_input_producer([ FILE ])
   features = read_and_decode(filename_queue)
   print(features["image/filename"])
   #image = tf.reshape(features[], tf.pack([height, width, 3]))
   #image.set_shape([720,720,3])
   #init_op = tf.initialize_all_variables()
   #sess.run(init_op)
   #coord = tf.train.Coordinator()
   #threads = tf.train.start_queue_runners(coord=coord)
   #for i in range(2053):
     #example, l = sess.run([image, label])
     #img = Image.fromarray(example, 'RGB')
     #img.save( "output/" + str(i) + '-train.png')

     #print (example,l)
   #coord.request_stop()
   #coord.join(threads)

get_all_records('../Tfdata/train.record')

NameError: global name 'file_name' is not defined

http://machinelearninguru.com/deep_learning/tensorflow/basics/tfrecord/tfrecord.html